In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from __future__ import division
from math import sqrt
import yaml
import time
from shapely.geometry import Point, Polygon, LineString, box
from environment import Environment, plot_environment, plot_line, plot_poly
from PathPlanner import PathPlanner

In [11]:
def draw_results(algo_name, path, V, E, env, bounds, object_radius, resolution, start_pose, goal_region, elapsed_time):
    path_length = 0.0
    for i in range(len(path)-1):
        path_length += euclidian_dist(path[i], path[i+1])
    title = algo_name + "\nВершин: " + str(len(V)) + " Препятствий: " + str(len(env.obstacles)) + " Число рёбер: " + str(len(path)) + "\nДлина пути: " + str(path_length) + "\nВремя работы (с)= " + str(elapsed_time)
    env_plot = plot_environment(env, bounds)
    env_plot.set_title(title)
    plot_poly(env_plot, goal_region, 'green')
    buffered_start_vertex = Point(start_pose).buffer(object_radius, resolution)
    plot_poly(env_plot, buffered_start_vertex, 'red')
    for edge in E:
        line = LineString([edge[0][:2], edge[1][:2]])
        plot_line(env_plot, line)
    for i in range(len(path)):
        path[i] = path[i][:2]
    plot_path(env_plot, path, object_radius)

def euclidian_dist(point1, point2):
    return sqrt((point2[0] - point1[0])**2 + (point2[1] - point1[1])**2)

def plot_path(env_plot, path, object_radius):
    line = LineString(path)
    x, y = line.xy
    env_plot.plot(x, y, color='red', linewidth=3, solid_capstyle='round', zorder=1)

In [12]:
def RRT(environment, bounds, start_pose, goal_region, car_radius, steer_distance, num_iterations, curve_type, resolution=3):
    start_time = time.time()
    a = PathPlanner()
    path, V, E = a.path(environment, bounds, start_pose, goal_region, car_radius, steer_distance, num_iterations, curve_type, resolution, RRT_Flavour="RRT")
    elapsed_time = time.time() - start_time
    if path:
        draw_results("RRT", path, V, E, environment, bounds, car_radius, resolution, start_pose, goal_region, elapsed_time)
    return path, V, E

def RRTStar(environment, bounds, start_pose, goal_region, car_radius, steer_distance, num_iterations, curve_type, resolution=3):
    start_time = time.time()
    a = PathPlanner()
    path, V, E = a.path(environment, bounds, start_pose, goal_region, car_radius, steer_distance, num_iterations, curve_type, resolution, RRT_Flavour="RRT*")
    elapsed_time = time.time() - start_time
    if path:
        draw_results("RRT*", path, V, E, environment, bounds, car_radius, resolution, start_pose, goal_region, elapsed_time)
    return path, V, E

In [ ]:
environment = Environment('example.yaml')
bounds = (-2, -3, 12, 8)
start_pose = (0, 0)
goal_region = Polygon([(10,5), (10,6), (11,6), (11,5)])
car_radius = 0.3
steer_distance = 0.3
num_iterations = 10000
resolution = 3
curve_type = "steer"

path, V, E = RRT(environment, bounds, start_pose, goal_region, car_radius, steer_distance, num_iterations, curve_type, resolution)
path, V, E = RRTStar(environment, bounds, start_pose, goal_region, car_radius, steer_distance, num_iterations, curve_type, resolution)